In [ ]:
%load_ext autoreload

In [ ]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import sos_iris_certifier.visualizations_utils as viz_utils
from sos_iris_certifier.iris_plant_visualizer import IrisPlantVisualizer

In [ ]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram
import meshcat

# Build plant

In [ ]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()


# Setup meshcat visualization

In [ ]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)

# Build Certified Iris Region Object

In [ ]:
# options for running non-linear optimizer to obtain good initial starting regions
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_ineqs': -1 # don't constrain the maximum number of inequalities. Let SNOPT figure it out
}

In [ ]:
q_star = np.zeros(3) # point around which we take the stereographic projection
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, q_star = q_star, **iris_kwargs)

In [ ]:
seed_points_q = np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ])
seed_points_t = np.tan(seed_points_q/2)

#
regions, ellipses = iris_generator.generate_initial_regions(seed_points_t)


In [ ]:
trajectory_start = seed_points_t[0,:]
trajectory_end = seed_points_t[3,:]

In [ ]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions, ellipses)
    visualizer.plot_seedpoints(seed_points_t)
    visualizer.visualize_collision_constraint(N = 30)
    

## Certify the regions

In [ ]:
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

In [ ]:
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

In [ ]:
# uncomment to visualize planes (TODO)
# visualizer.region_to_collision_pair_to_plane_dictionary = iris_generator.linesearch_regions_to_certificates_by_collision_pair_map

In [ ]:
do_alt_cert = False
if do_alt_cert:
    iris_generator.certify_and_adjust_regions_by_bilinear_alternation(max_iter = 100, 
                                                                 stopped_improving_count_threshold = 5,
                                                                 not_improving_threshold = 1e-4)

In [ ]:
# plot new regions
if do_viz:
    if do_alt_cert:
        visualizer.plot_regions(iris_generator.alternation_search_regions, region_suffix = '_new_by_alternation')
    if do_linesearch_cert:
        visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')
    

In [ ]:
from sos_iris_certifier.spp import solveBsplineTrajectory
# Solve path planning
start_time = time.time()
# trajectory through linesearch_regions

traj, result, spp, hard_result = solveBsplineTrajectory(trajectory_start, trajectory_end, 
                                                        iris_generator.linesearch_regions,
                                                        max_velocity=0.3*np.ones(plant.num_positions()))



In [ ]:
visualizer.draw_traj_s_space(traj, 100, 'iris')

In [ ]:
substeps = 1000
its = 10
visualizer.animate_s(traj, substeps, its*substeps)

In [ ]:
# TODO(Alex.Amice) fix the plane plotting
# visualizer.region_to_collision_pair_to_plane_dictionary = iris_generator.linesearch_regions_to_certificates_by_collision_pair_map